# EDA, Pandas, and the Grammar of Data

---

We'd like a data structure that can represent the columns in the data above by their name. In particular, we want a structure that can easily store variables of different types, that stores column names, and that we can reference by column name as well as by indexed position.  And it would be nice this data structure came with built-in functions that we can use to manipulate it. 

Pandas is a package/library that does all of this!  The library is built on top of numpy.  There are two basic pandas objects, *series* and *dataframes*, which can be thought of as enhanced versions of 1D and 2D numpy arrays, respectively.  Indeed Pandas attempts to keep all the efficiencies that `numpy` gives us.

For reference, here is a useful pandas [cheat sheet](https://drive.google.com/folderview?id=0ByIrJAE4KMTtaGhRcXkxNHhmY2M&usp=sharing) and the pandas [documentation](https://pandas.pydata.org/pandas-docs/stable/).

## The basic EDA workflow

Below is a basic checklist for the early stages of exploratory data analysis in Python. While not universally applicable, the rubric covers patterns which recur in several data analysis contexts, so useful to keep it in mind when encountering a new dataset.

The basic workflow (enunciated in this form by Chris Beaumont, the first Head TF of cs109 ever) is as follows:

1. **Build** a Dataframe from the data (ideally, put all data in this object)
2. **Clean** the Dataframe. It should have the following properties:
    - Each row describes a single object
    - Each column describes a property of that object
    - Columns are numeric whenever appropriate
    - Columns contain atomic properties that cannot be further decomposed
    
3. Explore **global properties**. Use histograms, scatter plots, and aggregation functions to summarize the data.
4. Explore **group properties**. Use groupby, queries, and small multiples to compare subsets of the data.

This process transforms the data into a format which is easier to work with, gives you a basic overview of the data's properties, and likely generates several questions for you to follow-up on in subsequent analysis.

In [1]:
# The %... is an iPython thing, and is not part of the Python language.
# In this case we're just telling the plotting library to draw things on
# the notebook, instead of on a separate window.
%matplotlib inline
# See all the "as ..." contructs? They're just aliasing the package names.
# That way we can call methods like plt.plot() instead of matplotlib.pyplot.plot().
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

### Building a dataframe

The easiest way to build a dataframe is simply to read in a CSV file. We saw an example of this in lab, and we shall see 2 examples here. We'll also see how we may combine multiple data sources into a larger dataframe.

This example is adapted from: https://github.com/tthibo/SQL-Tutorial

The first 3 lines of the file (`!head -3 data/candidates.txt` on mac/unix) look like this.

```
id|first_name|last_name|middle_name|party
33|Joseph|Biden||D
36|Samuel|Brownback||R
```

In [2]:
dfcand=pd.read_csv("./data/candidates.txt", sep='|')
dfcand.head()

,id,first_name,last_name,middle_name,party
0,33,Joseph,Biden,NaN,D
1,36,Samuel,Brownback,NaN,R
2,34,Hillary,Clinton,R.,D
3,39,Christopher,Dodd,J.,D
4,26,John,Edwards,NaN,D


A pandas dataframe is a set of columns pasted together into a spreadsheet, as shown in the schematic below, which is taken from the cheatsheet above.  The columns in pandas are called *series* objects.

![](images/pandastruct.png)

All the columns in this dataframe:

In [3]:
dfcand.columns

Index(['id', 'first_name', 'last_name', 'middle_name', 'party'], dtype='object')

And the types of these columns:

In [4]:
dfcand.dtypes

id              int64
first_name     object
last_name      object
middle_name    object
party          object
dtype: object

Access to a particular column can be obtained by treating the column name as an "attribute" of the dataframe:

In [5]:
dfcand.first_name

0          Joseph
1          Samuel
2         Hillary
3     Christopher
4            John
5         Rudolph
6            Mike
7            Mike
8          Duncan
9          Dennis
10           John
11         Barack
12            Ron
13           Bill
14           Mitt
15            Tom
16           Fred
Name: first_name, dtype: object

But Pandas supports a dictionary like access to columns. This is very useful when column names have spaces: Python variables cannot have spaces in them.

In [6]:
dfcand['first_name']

0          Joseph
1          Samuel
2         Hillary
3     Christopher
4            John
5         Rudolph
6            Mike
7            Mike
8          Duncan
9          Dennis
10           John
11         Barack
12            Ron
13           Bill
14           Mitt
15            Tom
16           Fred
Name: first_name, dtype: object

We can also get sub-dataframes by choosing a set of series. We pass a list of the columns we want as "dictionary keys" to the dataframe.

In [7]:
columns_i_want=['first_name', 'last_name']
dfcand[columns_i_want]

,first_name,last_name
0,Joseph,Biden
1,Samuel,Brownback
2,Hillary,Clinton
3,Christopher,Dodd
4,John,Edwards
5,Rudolph,Giuliani
6,Mike,Gravel
7,Mike,Huckabee
8,Duncan,Hunter
9,Dennis,Kucinich


### Categoricals

Even though `party` is a string, it takes on only a finite set of values, 'D', and 'R'. We can model this:

In [8]:
dfcand.party.unique()

array(['D', 'R'], dtype=object)

In [9]:
dfcand['party'] = dfcand['party'].astype("category")

In [10]:
dfcand.dtypes

id                int64
first_name       object
last_name        object
middle_name      object
party          category
dtype: object

In [11]:
dfcand.head()

,id,first_name,last_name,middle_name,party
0,33,Joseph,Biden,NaN,D
1,36,Samuel,Brownback,NaN,R
2,34,Hillary,Clinton,R.,D
3,39,Christopher,Dodd,J.,D
4,26,John,Edwards,NaN,D


In [12]:
dfcand.party.describe()

count     17
unique     2
top        R
freq       9
Name: party, dtype: object

In [13]:
dfcand.party.cat.categories

Index(['D', 'R'], dtype='object')

In [14]:
dfcand.party.cat.ordered

False

Keep in mind that this is a relatively new feature of Pandas. You dont need to do this, but might find it useful to keep your types straight. (Using categoricals in machine learning algorithms is more complex and usually involves a process called One Hot Encoding)

### Another piece of data

This is a file of people who have contributed money to candidates:
(`!head -3 data/contributors_with_candidate_id.txt`)

```
id|last_name|first_name|middle_name|street_1|street_2|city|state|zip|amount|date|candidate_id
|Agee|Steven||549 Laurel Branch Road||Floyd|VA|24091|500.00|2007-06-30|16
|Ahrens|Don||4034 Rennellwood Way||Pleasanton|CA|94566|250.00|2007-05-16|16
```

In [15]:
dfcwci=pd.read_csv("./data/contributors_with_candidate_id.txt", sep="|")
dfcwci.head()

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,NaN,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
2,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
3,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
4,NaN,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16


### Cleaning Data

most of the techniques you will learn about in Pandas are all about getting data in a form that can be used for further analysis. Cleaning usually means dealing with missing values, transforming types appropriately, and taking care of data integrity. But we'll lump everything required to transform data to a form appropriate for analysis **cleaning**, even if what we are doing is for example, combining multiple data sets, or producing processed data from raw data.

Lets start with some regular cleaning:

In [16]:
del dfcwci['id']
dfcwci.head()

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16


We can see the size of our data

In [17]:
dfcwci.shape, dfcand.shape

((175, 11), (17, 5))

We will do some more cleaning soon but let us see the EDA process as sliced from another angle: data transformation, used both for cleaning and for seeing single column and multiple column properties in the data.

## Data transformation: Single Table Verbs

Let us now focus on core data manipulation commands. These are *universal across systems, and by identifying them, we can quickly ask how to do these* when we encounter a new system.

See https://gist.github.com/TomAugspurger/6e052140eaa5fdb6e8c0/ which has a comparison of r/dplyr and pandas. I stole and modified this table from there:

``dplyr`` has a small set of nicely defined verbs, which Hadley Wickham has used to identify core data manipulation commands. Here are listed the closest SQL and Pandas verbs, so we can see the universality of these manipulations.


<table>
  <tr>
    <th><b>VERB</b></th>
    <th><b>dplyr</b></th>
    <th><b>pandas</b></th>
    <th><b>SQL</b></th>
  </tr>
  <tr>
    <td>QUERY/SELECTION</td>
    <td>filter() (and slice())</td>
    <td>query() (and loc[], iloc[])</td>
    <td>SELECT WHERE</td>
  </tr>
  <tr>
    <td>SORT</td>
    <td>arrange()</td>
    <td>sort()</td>
    <td>ORDER BY</td>
  </tr>
  <tr>
    <td>SELECT-COLUMNS/PROJECTION</td>
    <td>select() (and rename())</td>
    <td>[](__getitem__) (and rename())</td>
    <td>SELECT COLUMN</td>
  </tr>
  <tr>
    <td>SELECT-DISTINCT</td>
    <td>distinct()</td>
    <td>unique(),drop_duplicates()</td>
    <td>SELECT DISTINCT COLUMN</td>
  </tr>
  <tr>
    <td>ASSIGN</td>
    <td>mutate() (and transmute())</td>
    <td>assign</td>
    <td>ALTER/UPDATE</td>
  </tr>
  <tr>
    <td>AGGREGATE</td>
    <td>summarise()</td>
    <td>describe(), mean(), max()</td>
    <td>None, AVG(),MAX()</td>
  </tr>
  <tr>
    <td>SAMPLE</td>
    <td>sample_n() and sample_frac()</td>
    <td>sample()</td>
    <td>implementation dep, use RAND()</td>
  </tr>
  <tr>
    <td>GROUP-AGG</td>
    <td>group_by/summarize</td>
    <td>groupby/agg, count, mean</td>
    <td>GROUP BY</td>
  </tr>
  <tr>
    <td>DELETE</td>
    <td>?</td>
    <td>drop/masking</td>
    <td>DELETE/WHERE</td>
  </tr>
</table>


We'll tackle these one by one in Pandas, since these are data manipulations you will do all the time.

### QUERY

In [18]:
dfcwci.amount < 400

0      False
1       True
2       True
3       True
4       True
5      False
6      False
7       True
8      False
9      False
10     False
11     False
12      True
13     False
14      True
15     False
16     False
17      True
18      True
19     False
20     False
21     False
22      True
23     False
24     False
25      True
26      True
27      True
28     False
29      True
       ...  
145     True
146     True
147     True
148     True
149     True
150    False
151     True
152     True
153     True
154     True
155    False
156    False
157     True
158     True
159    False
160     True
161     True
162     True
163    False
164    False
165     True
166     True
167     True
168    False
169    False
170     True
171    False
172     True
173    False
174    False
Name: amount, Length: 175, dtype: bool

This gives us Trues and Falses. Such a series is called a **mask**.  A mask  is the basis of filtering. We can do:

In [19]:
dfcwci[dfcwci.amount < 400].head()

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16
7,Aldridge,Brittni,NaN,"808 Capitol Square Place, SW",NaN,Washington,DC,20024,250.0,2007-06-06,16


Notice that the dataframe has been filtered down to only include those contributions with amount < 400. The rows with `False` in the mask have been eliminated, and those with `True` in the mask have been kept.

In [20]:
np.sum(dfcwci.amount < 400), np.mean(dfcwci.amount < 400)

(132, 0.75428571428571434)

Why did that work? The booleans are coerced to integers as below:

In [21]:
1*True, 1*False

(1, 0)

Or directly, in Pandas, which works since the comparison is a pandas Series.

In [22]:
(dfcwci.amount < 400).mean()

0.75428571428571434

You can combine queries. Note that we use Python's `&` operator instead of `and`. This is because we are "Boolean AND"ing masks to get a series of True's And Falses.

In [23]:
dfcwci[(dfcwci.state=='VA') & (dfcwci.amount < 400)]

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
27,Buckheit,Bruce,NaN,8904 KAREN DR,NaN,FAIRFAX,VA,220312731,100.00,2007-09-19,20
77,Ranganath,Anoop,NaN,2507 Willard Drive,NaN,Charlottesville,VA,22903,-100.00,2008-04-21,32
88,Perreault,Louise,NaN,503 Brockridge Hunt Drive,NaN,Hampton,VA,23666,-34.08,2008-04-21,32
145,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,NaN,FAIRFAX,VA,220308440,50.00,2007-09-30,35


Here is another way to write the query:

In [24]:
dfcwci.query("state=='VA' & amount < 400")

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
27,Buckheit,Bruce,NaN,8904 KAREN DR,NaN,FAIRFAX,VA,220312731,100.00,2007-09-19,20
77,Ranganath,Anoop,NaN,2507 Willard Drive,NaN,Charlottesville,VA,22903,-100.00,2008-04-21,32
88,Perreault,Louise,NaN,503 Brockridge Hunt Drive,NaN,Hampton,VA,23666,-34.08,2008-04-21,32
145,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,NaN,FAIRFAX,VA,220308440,50.00,2007-09-30,35


For cleaning, we might want to use this querying ability

In [25]:
dfcwci[dfcwci.state.isnull()]

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
125,BOURNE,TRAVIS,NaN,LAGE KAART 77,NaN,BRASSCHATT,NaN,2930,-500.0,2008-11-20,35


Or the opposite, which is probably more useful in making the selection:

In [26]:
dfcwci[dfcwci.state.notnull()].head()

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16


For categoricals you can use `isin`. You can use Boolean not on the mask to implement not in.

In [27]:
dfcwci[dfcwci.state.isin(['VA','WA'])].head(10)

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.00,2007-06-30,16
27,Buckheit,Bruce,NaN,8904 KAREN DR,NaN,FAIRFAX,VA,220312731,100.00,2007-09-19,20
62,BURKE,SUZANNE,M.,3401 EVANSTON,NaN,SEATTLE,WA,981038677,-700.00,2008-03-05,22
77,Ranganath,Anoop,NaN,2507 Willard Drive,NaN,Charlottesville,VA,22903,-100.00,2008-04-21,32
88,Perreault,Louise,NaN,503 Brockridge Hunt Drive,NaN,Hampton,VA,23666,-34.08,2008-04-21,32
100,Aaronson,Rebecca,NaN,2000 Village Green Dr Apt 12,NaN,Mill Creek,WA,980125787,100.00,2008-02-08,34
106,Aaronson,Rebecca,NaN,2000 Village Green Dr Apt 12,NaN,Mill Creek,WA,980125787,100.00,2008-02-14,34
145,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,NaN,FAIRFAX,VA,220308440,50.00,2007-09-30,35


And you can chain queries thus.

In [28]:
dfcwci.query("10 <= amount <= 50").head(10)

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
18,Ardle,William,NaN,412 Dakota Avenue,NaN,Springfield,OH,45504,50.0,2007-06-28,16
25,Buckler,Steve,NaN,24351 Armada Dr,NaN,Dana Point,CA,926291306,50.0,2007-07-30,20
26,Buckler,Steve,NaN,24351 Armada Dr,NaN,Dana Point,CA,926291306,25.0,2007-08-16,20
34,Buck,Barbara,NaN,1780 NE 138th St,NaN,North Miami,FL,331811316,50.0,2007-09-13,20
35,Buck,Barbara,NaN,1780 NE 138th St,NaN,North Miami,FL,331811316,50.0,2007-07-19,20
38,Buchanek,Elizabeth,NaN,7917 Kentbury Dr,NaN,Bethesda,MD,208144615,50.0,2007-09-30,20
49,Harrison,Ryan,NaN,2247 3rd St,NaN,La Verne,CA,917504918,25.0,2007-07-26,20
101,Aarons,Elaine,NaN,481 Buck Island Rd Apt 17A,APT 17A,West Yarmouth,MA,26733300,25.0,2008-02-26,34
104,Aaron,Shirley,NaN,101 Cherry Ave,NaN,Havana,FL,323331311,50.0,2008-02-29,34


### SORT

In [29]:
dfcwci.sort_values(by="amount").head(10)

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
90,Kazor,Christopher,M,707 Spindletree ave,NaN,Naperville,IL,60565,-2592.0,2008-04-21,32
72,BRUNO,JOHN,NaN,10136 WINDERMERE CHASE BLVD.,NaN,GOTHA,FL,347344707,-2300.0,2008-03-06,22
64,BURKE,DONALD,J.,12 LOMPOC,NaN,RANCHO SANTA MARGA,CA,926881817,-2300.0,2008-03-11,22
73,BRUNO,IRENE,NaN,10136 WINDERMERE CHASE BLVD.,NaN,GOTHA,FL,347344707,-2300.0,2008-03-06,22
74,BROWN,TIMOTHY,J.,26826 MARLOWE COURT,NaN,STEVENSON RANCH,CA,913811020,-2300.0,2008-03-06,22
58,BURTON,GLENN,M.,4404 CHARLESTON COURT,NaN,TAMPA,FL,336092620,-2300.0,2008-03-05,22
57,BURTON,STEVEN,G.,9938 DEER CREEK DRIVE,NaN,TAMPA,FL,33647,-2300.0,2008-03-05,22
84,Uihlein,Richard,NaN,1396 N Waukegan Rd,NaN,Lake Forest,IL,600451147,-2300.0,2008-04-21,32
56,BURTON,SUSAN,NaN,9338 DEER CREEK DRIVE,NaN,TAMPA,FL,336472286,-2300.0,2008-03-05,22
55,BUSH,ERIC,NaN,P.O. BOX 61046,NaN,DENVER,CO,802061046,-2300.0,2008-03-06,22


In [30]:
dfcwci[dfcwci.amount < 0]

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
29,Buckel,Linda,NaN,PO Box 683130,NaN,Park City,UT,840683130,-2300.00,2007-08-14,20
40,Buchanan,John,NaN,2025 NW 29th Rd,NaN,Boca Raton,FL,334316303,-500.00,2007-09-24,20
43,Buchanan,John,NaN,2025 NW 29th Rd,NaN,Boca Raton,FL,334316303,-700.00,2007-08-28,20
50,BYNUM,HERBERT,NaN,332 SUNNYSIDE ROAD,NaN,TAMPA,FL,336177249,-500.00,2008-03-10,22
51,BYINGTON,MARGARET,E.,2633 MIDDLEBORO LANE N.E.,NaN,GRAND RAPIDS,MI,495061254,-2300.00,2008-03-03,22
52,BYERS,BOB,A.,13170 TELFAIR AVENUE,NaN,SYLMAR,CA,913423573,-2300.00,2008-03-07,22
53,BYERS,AUDREY,NaN,2658 LADBROOK WAY,NaN,THOUSAND OAKS,CA,913615073,-200.00,2008-03-07,22
54,BUSH,KRYSTIE,NaN,P.O. BOX 61046,NaN,DENVER,CO,802061046,-2300.00,2008-03-06,22
55,BUSH,ERIC,NaN,P.O. BOX 61046,NaN,DENVER,CO,802061046,-2300.00,2008-03-06,22
56,BURTON,SUSAN,NaN,9338 DEER CREEK DRIVE,NaN,TAMPA,FL,336472286,-2300.00,2008-03-05,22


In [31]:
dfcwci.sort_values(by="amount", ascending=False).head(10)

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
30,Buckel,Linda,NaN,PO Box 683130,NaN,Park City,UT,840683130,4600.0,2007-08-14,20
159,ABATE,MARIA,ELENA,1291 NIGHTINGALE AVENUE,NaN,MIAMI SPRINGS,FL,331663832,2600.0,2008-01-25,37
15,Anthony,John,NaN,211 Long Island Drive,NaN,Hot Springs,AR,71913,2300.0,2007-06-12,16
33,Buck,Blaine,M,45 Eaton Ave,NaN,Camden,ME,48431752,2300.0,2007-09-30,20
28,Buckel,Linda,NaN,PO Box 683130,NaN,Park City,UT,840683130,2300.0,2007-08-14,20
21,Baker,David,NaN,2550 Adamsbrooke Drive,NaN,Conway,AR,72034,2300.0,2007-04-11,16
13,Altes,R.D.,NaN,8600 Moody Road,NaN,Fort Smith,AR,72903,2300.0,2007-06-21,16
135,ABRAMOWITZ,NIRA,NaN,411 HARBOR ROAD,NaN,SOUTHPORT,CT,68901376,2300.0,2007-09-14,35
5,Akin,Mike,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,1500.0,2007-05-18,16
174,ABRAHAM,SALEM,A.,P.O. BOX 7,NaN,CANADIAN,TX,790140007,1300.0,2008-01-30,37


### SELECT-COLUMNS

In [32]:
dfcwci[['first_name', 'amount']].head(10)

,first_name,amount
0,Steven,500.0
1,Don,250.0
2,Don,50.0
3,Don,100.0
4,Charles,100.0
5,Mike,1500.0
6,Rebecca,500.0
7,Brittni,250.0
8,John D.,1000.0
9,John D.,1300.0


### SELECT-DISTINCT

Selecting a distinct set is useful for cleaning. Here, we might wish to focus on contributors rather than contributions and see how many distinct contributors we have. Of-course we might be wrong, some people have identical names. 

In [33]:
dfcwci[['last_name','first_name']].count()

last_name     175
first_name    175
dtype: int64

In [34]:
dfcwci[['last_name','first_name']].drop_duplicates().count()

last_name     126
first_name    126
dtype: int64

In [35]:
dfcwci[['last_name','first_name']].drop_duplicates().head(10)

,last_name,first_name
0,Agee,Steven
1,Ahrens,Don
4,Akin,Charles
5,Akin,Mike
6,Akin,Rebecca
7,Aldridge,Brittni
8,Allen,John D.
10,Allison,John W.
11,Allison,Rebecca
13,Altes,R.D.


### ASSIGN

Assignment to a new column is easy.

In [36]:
dfcwci['name']=dfcwci['last_name']+", "+dfcwci['first_name']
dfcwci.head(10)

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16,"Agee, Steven"
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16,"Ahrens, Don"
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16,"Ahrens, Don"
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16,"Ahrens, Don"
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16,"Akin, Charles"
5,Akin,Mike,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,1500.0,2007-05-18,16,"Akin, Mike"
6,Akin,Rebecca,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,500.0,2007-05-18,16,"Akin, Rebecca"
7,Aldridge,Brittni,NaN,"808 Capitol Square Place, SW",NaN,Washington,DC,20024,250.0,2007-06-06,16,"Aldridge, Brittni"
8,Allen,John D.,NaN,1052 Cannon Mill Drive,NaN,North Augusta,SC,29860,1000.0,2007-06-11,16,"Allen, John D."
9,Allen,John D.,NaN,1052 Cannon Mill Drive,NaN,North Augusta,SC,29860,1300.0,2007-06-29,16,"Allen, John D."


In [37]:
dfcwci.assign(ucname=dfcwci.last_name+":"+dfcwci.first_name).head(10)

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name,ucname
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16,"Agee, Steven",Agee:Steven
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16,"Ahrens, Don",Ahrens:Don
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16,"Ahrens, Don",Ahrens:Don
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16,"Ahrens, Don",Ahrens:Don
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16,"Akin, Charles",Akin:Charles
5,Akin,Mike,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,1500.0,2007-05-18,16,"Akin, Mike",Akin:Mike
6,Akin,Rebecca,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,500.0,2007-05-18,16,"Akin, Rebecca",Akin:Rebecca
7,Aldridge,Brittni,NaN,"808 Capitol Square Place, SW",NaN,Washington,DC,20024,250.0,2007-06-06,16,"Aldridge, Brittni",Aldridge:Brittni
8,Allen,John D.,NaN,1052 Cannon Mill Drive,NaN,North Augusta,SC,29860,1000.0,2007-06-11,16,"Allen, John D.",Allen:John D.
9,Allen,John D.,NaN,1052 Cannon Mill Drive,NaN,North Augusta,SC,29860,1300.0,2007-06-29,16,"Allen, John D.",Allen:John D.


Will the above command actually change `dfcwci`?. No, it produces a fresh dataframe.

#### What if we wanted to change an existing assignment?

In [38]:
dfcwci[dfcwci.state=='VA']

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.00,2007-06-30,16,"Agee, Steven"
27,Buckheit,Bruce,NaN,8904 KAREN DR,NaN,FAIRFAX,VA,220312731,100.00,2007-09-19,20,"Buckheit, Bruce"
77,Ranganath,Anoop,NaN,2507 Willard Drive,NaN,Charlottesville,VA,22903,-100.00,2008-04-21,32,"Ranganath, Anoop"
88,Perreault,Louise,NaN,503 Brockridge Hunt Drive,NaN,Hampton,VA,23666,-34.08,2008-04-21,32,"Perreault, Louise"
145,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,NaN,FAIRFAX,VA,220308440,50.00,2007-09-30,35,"ABDELLA, THOMAS"


In [39]:
dfcwci.loc[dfcwci.state=='VA', 'name']

0           Agee, Steven
27       Buckheit, Bruce
77      Ranganath, Anoop
88     Perreault, Louise
145      ABDELLA, THOMAS
Name: name, dtype: object

In [40]:
dfcwci.loc[dfcwci.state=='VA', 'name']="junk"

In [41]:
dfcwci.query("state=='VA'")

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.00,2007-06-30,16,junk
27,Buckheit,Bruce,NaN,8904 KAREN DR,NaN,FAIRFAX,VA,220312731,100.00,2007-09-19,20,junk
77,Ranganath,Anoop,NaN,2507 Willard Drive,NaN,Charlottesville,VA,22903,-100.00,2008-04-21,32,junk
88,Perreault,Louise,NaN,503 Brockridge Hunt Drive,NaN,Hampton,VA,23666,-34.08,2008-04-21,32,junk
145,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,NaN,FAIRFAX,VA,220308440,50.00,2007-09-30,35,junk


### Drop-Column

Real simple:

In [42]:
del dfcwci['name']

### AGGREGATE

In [43]:
dfcwci.describe()

,zip,amount,candidate_id
count,1.750000e+02,175.000000,175.000000
mean,3.780014e+08,3.418114,28.000000
std,3.628278e+08,1028.418999,7.823484
min,2.474000e+03,-2592.000000,16.000000
25%,9.336700e+04,-175.000000,20.000000
50%,3.233313e+08,100.000000,32.000000
75%,7.816946e+08,300.000000,35.000000
max,9.951532e+08,4600.000000,37.000000


In [44]:
dfcwci.amount.max()

4600.0

In [45]:
dfcwci[dfcwci.amount==dfcwci.amount.max()]

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
30,Buckel,Linda,NaN,PO Box 683130,NaN,Park City,UT,840683130,4600.0,2007-08-14,20


In [46]:
dfcwci[dfcwci.amount > dfcwci.amount.max() - 2300]

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
30,Buckel,Linda,NaN,PO Box 683130,NaN,Park City,UT,840683130,4600.0,2007-08-14,20
159,ABATE,MARIA,ELENA,1291 NIGHTINGALE AVENUE,NaN,MIAMI SPRINGS,FL,331663832,2600.0,2008-01-25,37


Aso `MIN`, `SUM`, `AVG`.

## Grouping using Pandas and split-apply-combine

In [47]:
grouped_by_state = dfcwci.groupby("state")
grouped_by_state

In [48]:
grouped_by_state.amount

How do we get access to these? Standard pandas functions distribute over the `groupby`, going one by one over the sub-dataframes or sub-series.  This is an example of a paradigm called split-apply-combine.

### GROUP-AGG

The fourth part of the EDA rubric is to look at properties of the sub-dataframes you get when you make groups. (We'll talk about the graphical aspects of this later). For instance, you might group contributions by state:

In [49]:
dfcwci.groupby("state").describe()

amount                                                                            candidate_id                                                         zip                                                                                               
       count         mean          std      min        25%       50%        75%     max        count       mean        std   min    25%   50%    75%   max count          mean           std          min           25%          50%           75%          max
state                                                                                                                                                                                                                                                          
AK       3.0   403.333333   100.166528   300.00   355.0000   410.000   455.0000   500.0          3.0  37.000000   0.000000  37.0  37.00  37.0  37.00  37.0   3.0  9.951532e+08  0.000000e+00  995153207.0  9.951532e+08  995153207.0  9.951532e+08  995153207.0
AR      12.0  1183.333333   775.574079   100.00   875.0000  1000.000  1700.0000  2300.0         12.0  16.000000   0.000000  16.0  16.00  16.0  16.00  16.0  12.0  7.206583e+04  4.250251e+02      71603.0  7.165500e+04      72033.5  7.222700e+04      72903.0
AZ       1.0   120.000000          NaN   120.00   120.0000   120.000   120.0000   120.0          1.0  37.000000        NaN  37.0  37.00  37.0  37.00  37.0   1.0  8.600111e+08           NaN  860011121.0  8.600111e+08  860011121.0  8.600111e+08  860011121.0
CA      23.0  -217.988261   942.102438 -2300.00  -175.0000    50.000   225.0000  1000.0         23.0  26.086957   8.151481  16.0  20.00  22.0  34.50  37.0  23.0  6.407113e+08  4.333269e+08      92127.0  9.456600e+04  913811020.0  9.260504e+08  941151435.0
CO       4.0 -1455.750000  1025.166125 -2300.00 -2300.0000 -1650.000  -805.7500  -223.0          4.0  27.750000   6.751543  22.0  22.00  27.0  32.75  35.0   4.0  6.013695e+08  4.008598e+08      80020.0  6.009768e+08  801668384.0  8.020610e+08  802061046.0
CT       1.0  2300.000000          NaN  2300.00  2300.0000  2300.000  2300.0000  2300.0          1.0  35.000000        NaN  35.0  35.00  35.0  35.00  35.0   1.0  6.890138e+07           NaN   68901376.0  6.890138e+07   68901376.0  6.890138e+07   68901376.0
DC       5.0  -309.982000   529.644175 -1000.00  -500.0000  -500.000   200.0900   250.0          5.0  20.400000   2.607681  16.0  20.00  22.0  22.00  22.0   5.0  1.600684e+08  8.946976e+07      20024.0  2.000525e+08  200052503.0  2.000525e+08  200164320.0
FL      30.0  -135.000000  1177.383862 -2300.00  -500.0000   100.000   500.0000  2600.0         30.0  26.766667   7.151963  20.0  20.00  22.0  34.00  37.0  30.0  2.990209e+08  1.015776e+08      33647.0  3.233313e+08  333063809.5  3.343163e+08  347344707.0
IA       1.0   250.000000          NaN   250.00   250.0000   250.000   250.0000   250.0          1.0  16.000000        NaN  16.0  16.00  16.0  16.00  16.0   1.0  5.026600e+04           NaN      50266.0  5.026600e+04      50266.0  5.026600e+04      50266.0
ID       1.0  -261.000000          NaN  -261.00  -261.0000  -261.000  -261.0000  -261.0          1.0  32.000000        NaN  32.0  32.00  32.0  32.00  32.0   1.0  8.364800e+04           NaN      83648.0  8.364800e+04      83648.0  8.364800e+04      83648.0
IL       6.0  -931.133333  1230.657981 -2592.00 -1936.2000  -472.400    12.5000   200.0          6.0  29.166667   6.645801  20.0  24.50  32.0  32.00  37.0   6.0  5.070114e+08  2.485720e+08      60565.0  6.005716e+08  603169827.0  6.062808e+08  628645116.0
KS       1.0  -330.000000          NaN  -330.00  -330.0000  -330.000  -330.0000  -330.0          1.0  32.000000        NaN  32.0  32.00  32.0  32.00  32.0   1.0  6.621500e+04           NaN      66215.0  6.621500e+04      66215.0  6.621500e+04      66215.0
KY       1.0  -200.000000          NaN  -200.00  -200.0000  -200.000  -200.0000  -200.0          1.0  22.000000        NaN  22.0  22.00  22.0  22.00  22.0   1.0  4.02

In [50]:
dfcwci.groupby("state").sum()

,zip,amount,candidate_id
state,,,
AK,2985459621,1210.00,111
AR,864790,14200.00,192
AZ,860011121,120.00,37
CA,14736360720,-5013.73,600
CO,2405477834,-5823.00,111
CT,68901376,2300.00,35
DC,800341853,-1549.91,102
FL,8970626520,-4050.00,803
IA,50266,250.00,16


In [51]:
dfcwci.groupby("state")['amount'].mean()

state
AK     403.333333
AR    1183.333333
AZ     120.000000
CA    -217.988261
CO   -1455.750000
CT    2300.000000
DC    -309.982000
FL    -135.000000
IA     250.000000
ID    -261.000000
IL    -931.133333
KS    -330.000000
KY    -200.000000
LA     650.000000
MA     -13.833333
MD     150.000000
ME     630.000000
MI    -253.000000
MN     107.333333
MO     100.000000
NC     500.000000
NH     -24.600000
NJ    -408.725000
NV     181.250000
NY    -809.312500
OH     112.500000
OK     266.666667
PA    -429.200000
RI     100.000000
SC     800.000000
TN     -25.000000
TX     220.582222
UT     459.090909
VA     103.184000
WA    -166.666667
Name: amount, dtype: float64

In [52]:
dfcwci.groupby("state")['amount'].apply(lambda x: np.std(x))

state
AK      81.785628
AR     742.555647
AZ       0.000000
CA     921.394361
CO     887.819907
CT       0.000000
DC     473.728151
FL    1157.594489
IA       0.000000
ID       0.000000
IL    1123.431895
KS       0.000000
KY       0.000000
LA     150.000000
MA     226.572591
MD     100.000000
ME     964.339152
MI    1034.522112
MN     113.611424
MO       0.000000
NC       0.000000
NH       0.000000
NJ     369.225000
NV     184.877493
NY     865.515126
OH     250.935749
OK     169.967317
PA     537.224869
RI       0.000000
SC     509.901951
TN       0.000000
TX     626.481285
UT    1636.316287
VA     209.965120
WA     377.123617
Name: amount, dtype: float64

The dictionary-like structure is more obvious in this method of iteration, but it does not do the combining part...

In [53]:
for k, v in dfcwci.groupby('state'):
    print("State", k, "mean amount", v.amount.mean(), "std", v.amount.std())

State AK mean amount 403.3333333333333 std 100.16652800877813
State AR mean amount 1183.3333333333333 std 775.574078676661
State AZ mean amount 120.0 std nan
State CA mean amount -217.9882608695652 std 942.1024379191479
State CO mean amount -1455.75 std 1025.16612474922
State CT mean amount 2300.0 std nan
State DC mean amount -309.98199999999997 std 529.6441745360747
State FL mean amount -135.0 std 1177.3838620520878
State IA mean amount 250.0 std nan
State ID mean amount -261.0 std nan
State IL mean amount -931.1333333333333 std 1230.6579811900083
State KS mean amount -330.0 std nan
State KY mean amount -200.0 std nan
State LA mean amount 650.0 std 212.13203435596427
State MA mean amount -13.833333333333334 std 248.19783775582468
State MD mean amount 150.0 std 141.4213562373095
State ME mean amount 630.0 std 1113.5229379466475
State MI mean amount -253.0 std 1156.630883212099
State MN mean amount 107.33333333333333 std 139.14500829470433
State MO mean amount 100.0 std nan
State NC mea

### DELETE

In [54]:
dfcwci.head()

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16


In-place drops

In [55]:
df2=dfcwci.copy()
df2.set_index('last_name', inplace=True)
df2.head()

,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
last_name,,,,,,,,,,
Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16


In [56]:
df2.drop(['Ahrens'], inplace=True)
df2.head()

,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
last_name,,,,,,,,,,
Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16
Akin,Mike,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,1500.0,2007-05-18,16
Akin,Rebecca,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,500.0,2007-05-18,16
Aldridge,Brittni,NaN,"808 Capitol Square Place, SW",NaN,Washington,DC,20024,250.0,2007-06-06,16


In [57]:
df2.reset_index(inplace=True)
df2.head()

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16
2,Akin,Mike,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,1500.0,2007-05-18,16
3,Akin,Rebecca,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,500.0,2007-05-18,16
4,Aldridge,Brittni,NaN,"808 Capitol Square Place, SW",NaN,Washington,DC,20024,250.0,2007-06-06,16


The recommended way to do it is to create a new dataframe. This might be impractical is things are very large.

In [58]:
dfcwci=dfcwci[dfcwci.last_name!='Ahrens']
dfcwci.head(10)

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16
5,Akin,Mike,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,1500.0,2007-05-18,16
6,Akin,Rebecca,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,500.0,2007-05-18,16
7,Aldridge,Brittni,NaN,"808 Capitol Square Place, SW",NaN,Washington,DC,20024,250.0,2007-06-06,16
8,Allen,John D.,NaN,1052 Cannon Mill Drive,NaN,North Augusta,SC,29860,1000.0,2007-06-11,16
9,Allen,John D.,NaN,1052 Cannon Mill Drive,NaN,North Augusta,SC,29860,1300.0,2007-06-29,16
10,Allison,John W.,NaN,P.O. Box 1089,NaN,Conway,AR,72033,1000.0,2007-05-18,16
11,Allison,Rebecca,NaN,3206 Summit Court,NaN,Little Rock,AR,72227,1000.0,2007-04-25,16
12,Allison,Rebecca,NaN,3206 Summit Court,NaN,Little Rock,AR,72227,200.0,2007-06-12,16


### LIMIT

In [59]:
dfcwci[0:3] # also see loc and iloc from the lab

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16
5,Akin,Mike,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,1500.0,2007-05-18,16


## Relationships: JOINs are Cartesian Products.

Finally, there are many occasions you will want to combine dataframes. We might want to see who contributed to Obama:

### Simple subselect

In [60]:
dfcand.head()

,id,first_name,last_name,middle_name,party
0,33,Joseph,Biden,NaN,D
1,36,Samuel,Brownback,NaN,R
2,34,Hillary,Clinton,R.,D
3,39,Christopher,Dodd,J.,D
4,26,John,Edwards,NaN,D


In [61]:
obamaid=dfcand.query("last_name=='Obama'")['id'].values[0]

In [62]:
obamacontrib=dfcwci.query("candidate_id==%i" % obamaid)
obamacontrib.head()

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
25,Buckler,Steve,NaN,24351 Armada Dr,NaN,Dana Point,CA,926291306,50.0,2007-07-30,20
26,Buckler,Steve,NaN,24351 Armada Dr,NaN,Dana Point,CA,926291306,25.0,2007-08-16,20
27,Buckheit,Bruce,NaN,8904 KAREN DR,NaN,FAIRFAX,VA,220312731,100.0,2007-09-19,20
28,Buckel,Linda,NaN,PO Box 683130,NaN,Park City,UT,840683130,2300.0,2007-08-14,20
29,Buckel,Linda,NaN,PO Box 683130,NaN,Park City,UT,840683130,-2300.0,2007-08-14,20


### Explicit INNER JOIN

This is the one you will want 90% of the time. It will only match keys that are present in both dataframes.

![inner join](http://pandas.pydata.org/pandas-docs/stable/_images/merging_merge_on_key_inner.png)

(from http://pandas.pydata.org/pandas-docs/stable/merging.html)

In [63]:
cols_wanted=['last_name_x', 'first_name_x', 'candidate_id', 'id', 'last_name_y']
dfcwci.merge(dfcand, left_on="candidate_id", right_on="id")[cols_wanted]

,last_name_x,first_name_x,candidate_id,id,last_name_y
0,Agee,Steven,16,16,Huckabee
1,Akin,Charles,16,16,Huckabee
2,Akin,Mike,16,16,Huckabee
3,Akin,Rebecca,16,16,Huckabee
4,Aldridge,Brittni,16,16,Huckabee
5,Allen,John D.,16,16,Huckabee
6,Allen,John D.,16,16,Huckabee
7,Allison,John W.,16,16,Huckabee
8,Allison,Rebecca,16,16,Huckabee
9,Allison,Rebecca,16,16,Huckabee


If the names of the columns you wanted to merge on were identical, you could simply say `on=id`, for example, rather than a `left_on` and a `right_on`.

### Outer JOIN

#### left outer (contributors on candidates)

![left outer](http://pandas.pydata.org/pandas-docs/stable/_images/merging_merge_on_key_left.png)

In [64]:
dfcwci.merge(dfcand, left_on="candidate_id", right_on="id", how="left")[cols_wanted]

,last_name_x,first_name_x,candidate_id,id,last_name_y
0,Agee,Steven,16,16,Huckabee
1,Akin,Charles,16,16,Huckabee
2,Akin,Mike,16,16,Huckabee
3,Akin,Rebecca,16,16,Huckabee
4,Aldridge,Brittni,16,16,Huckabee
5,Allen,John D.,16,16,Huckabee
6,Allen,John D.,16,16,Huckabee
7,Allison,John W.,16,16,Huckabee
8,Allison,Rebecca,16,16,Huckabee
9,Allison,Rebecca,16,16,Huckabee


#### right outer (contributors on candidates) = left outer (candidates on contributors)

![right outer](http://pandas.pydata.org/pandas-docs/stable/_images/merging_merge_on_key_right.png)

In [65]:
dfcwci.merge(dfcand, left_on="candidate_id", right_on="id", how="right")[cols_wanted]

,last_name_x,first_name_x,candidate_id,id,last_name_y
0,Agee,Steven,16.0,16,Huckabee
1,Akin,Charles,16.0,16,Huckabee
2,Akin,Mike,16.0,16,Huckabee
3,Akin,Rebecca,16.0,16,Huckabee
4,Aldridge,Brittni,16.0,16,Huckabee
5,Allen,John D.,16.0,16,Huckabee
6,Allen,John D.,16.0,16,Huckabee
7,Allison,John W.,16.0,16,Huckabee
8,Allison,Rebecca,16.0,16,Huckabee
9,Allison,Rebecca,16.0,16,Huckabee


#### full outer

![outer](http://pandas.pydata.org/pandas-docs/stable/_images/merging_merge_on_key_outer.png)

In [66]:
dfcwci.merge(dfcand, left_on="candidate_id", right_on="id", how="outer")[cols_wanted]

,last_name_x,first_name_x,candidate_id,id,last_name_y
0,Agee,Steven,16.0,16,Huckabee
1,Akin,Charles,16.0,16,Huckabee
2,Akin,Mike,16.0,16,Huckabee
3,Akin,Rebecca,16.0,16,Huckabee
4,Aldridge,Brittni,16.0,16,Huckabee
5,Allen,John D.,16.0,16,Huckabee
6,Allen,John D.,16.0,16,Huckabee
7,Allison,John W.,16.0,16,Huckabee
8,Allison,Rebecca,16.0,16,Huckabee
9,Allison,Rebecca,16.0,16,Huckabee


When to use which?

See this:

http://blog.codinghorror.com/a-visual-explanation-of-sql-joins/

## Useful Links

- http://sebastianraschka.com/Articles/sqlite3_database.html and  http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html#unique_indexes
- https://github.com/tthibo/SQL-Tutorial
- http://chrisalbon.com

And especially for R users:

- https://cran.r-project.org/web/packages/dplyr/vignettes/introduction.html
- https://gist.github.com/TomAugspurger/6e052140eaa5fdb6e8c0/